In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# 文件路径（替换为你的实际路径）
file_path = r"D:\朱思潮\编程\学习编程\PYTHON\学习python\数据分析与机器学习\基本算法语法\机器学习\RNN循环\秒速5厘米.txt"

# 读取文件内容
with open(file_path, "r", encoding="utf-8") as f:  # "r"表示读取，encoding指定为utf-8处理中文
    text = f.read()  # 读取全部内容到text变量
text = text.replace("  ", " ").strip()


In [3]:
# 提取所有唯一汉字（包括标点）
chars = sorted(list(set(text)))  # 所有不重复的汉字+标点
char2idx = {c: i for i, c in enumerate(chars)}  # 汉字→索引
idx2char = {i: c for i, c in enumerate(chars)}  # 索引→汉字
n_chars = len(chars)  # 汉字总数（词汇表大小，中文通常比英文大）
print(f"汉字总数：{n_chars}")  # 示例文本可能有几十到几百个汉字

汉字总数：2439


In [4]:
seq_length = 50  # 中文序列长度可适当缩短（因汉字信息密度更高）
dataX = []
dataY = []

# 将文本转换为索引序列
text_idx = [char2idx[c] for c in text]

# 生成输入-目标对
for i in range(len(text_idx) - seq_length):
    dataX.append(text_idx[i:i+seq_length])  # 输入：[i, i+seq_length)
    dataY.append(text_idx[i+1:i+1+seq_length])  # 目标：下一个字符序列

# 转换为张量
dataX = torch.tensor(dataX)  # 形状：(num_sequences, seq_length)
dataY = torch.tensor(dataY)  # 形状：(num_sequences, seq_length)

In [5]:
class ChineseTextGenerator(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)  # 汉字嵌入层
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)  # 输出层预测下一个汉字
    
    def forward(self, x, hidden):
        x = self.embedding(x)  # (batch_size, seq_length, hidden_size)
        out, hidden = self.lstm(x, hidden)
        out = out.reshape(-1, self.hidden_size)  # 展平为(batch*seq, hidden)
        out = self.fc(out)  # (batch*seq, vocab_size)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return (weight.new_zeros(self.n_layers, batch_size, self.hidden_size),
                weight.new_zeros(self.n_layers, batch_size, self.hidden_size))

In [6]:
hidden_size = 256  # 比英文稍大，适配更多汉字
n_layers = 2
batch_size = 16
n_epochs = 10  # 中文训练可能需要更多轮次

# 初始化模型、损失函数、优化器
model = ChineseTextGenerator(vocab_size=n_chars, hidden_size=hidden_size, n_layers=n_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    permutation = torch.randperm(len(dataX))
    
    for i in range(0, len(dataX), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x = dataX[indices]
        batch_y = dataY[indices]
        batch_size_current = batch_x.size(0)
        
        hidden = model.init_hidden(batch_size_current)
        output, hidden = model(batch_x, hidden)
        loss = criterion(output, batch_y.view(-1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / (len(dataX) // batch_size)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")

KeyboardInterrupt: 

In [ ]:
def generate_chinese(model, start_str, length=200, temperature=0.8):
    """生成中文文本"""
    model.eval()
    with torch.no_grad():
        # 初始输入：中文起始字符串（如"黛玉"）
        input_seq = torch.tensor([char2idx[c] for c in start_str]).unsqueeze(0)  # (1, len(start_str))
        hidden = model.init_hidden(1)
        generated = start_str
        
        for _ in range(length):
            output, hidden = model(input_seq, hidden)
            output = output[-1, :] / temperature  # 最后一个字符的输出
            probs = torch.softmax(output, dim=0)
            next_idx = torch.multinomial(probs, 1).item()  # 采样下一个汉字
            next_char = idx2char[next_idx]
            generated += next_char
            
            # 更新输入：仅保留最新的字符（长度为1）
            input_seq = torch.tensor([[next_idx]]).unsqueeze(0)
    
    return generated

# 示例：从"樱花"开始生成200字
print(generate_chinese(model, start_str="樱花", length=2000, temperature=0.7))

NameError: name 'model' is not defined